In [42]:
import pandas as pd
import numpy as np
import random 

#data1 = pd.read_csv("data/Amazon_ratings.csv") # 데이터 사이즈 줄이기 전 데이터
#data2 = pd.read_csv("data/Amazon_reviews.csv")
ratings = pd.read_csv("/home/dan/@Coursework2/recsys/Final_project/data/Amazon_ratings_sizedown.csv") #데이터 사이즈 줄인 후 데이터
reviews = pd.read_csv("/home/dan/@Coursework2/recsys/Final_project/data/Amazon_reviews_sizedown.csv")

from datetime import datetime
# timestamp -> datetime으로 변경
ratings['time'] =  pd.to_datetime(ratings['timestamp'], unit='s')

# 연도, 월, 요일 칼럼 만들기
ratings['year'] = ratings['time'].dt.year
ratings['month'] = ratings['time'].dt.month
ratings['weekday'] = ratings['time'].dt.weekday

winter = [12,1,2]
spring = [3,4,5]
summer = [6,7,8]
fall = [9,10,11]

ratings['season'] = ratings['month']


for i in range(len(ratings)):
    if ratings['season'][i] in winter:
        ratings['season'][i] = 0
        
    elif ratings['season'][i] in spring:
        ratings['season'][i] = 1
        
    elif ratings['season'][i] in summer:
        ratings['season'][i] = 2
        
    elif ratings['season'][i] in fall:
        ratings['season'][i] = 3
        
ratings = ratings.drop('time', axis=1)

df = ratings.copy()


from sklearn.utils import shuffle
# train test 분리
TRAIN_SIZE = 0.7
ratings = shuffle(ratings, random_state=12)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

ratings = ratings_train.pivot(index = 'user_id', columns ='item_id', values = 'rating').fillna(0)

/tmp/ipykernel_42084/2072563075.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.read_csv("/home/dan/@Coursework2/recsys/Final_project/data/Amazon_reviews_sizedown.csv")
/tmp/ipykernel_42084/2072563075.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['season'][i] = 0
/tmp/ipykernel_42084/2072563075.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['season'][i] = 2
/tmp/ipykernel_42084/2072563075.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

In [43]:
###################################### MF ##################################################
# Predictions using MF ###########################################################################
class NEW_MF():
    # Initializing the object
    def __init__(self, ratings, K, alpha, beta, iterations, tolerance=0.005, verbose=True):
        self.R = np.array(ratings)
        # user_id, movie_id를 R의 index와 매칭하기 위한 dictionary 생성
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings):
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)        
        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)
        # 다른 변수 초기화
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.tolerance = tolerance
        self.verbose = verbose

    # 테스트 셋을 선정하는 메소드 
    def set_test(self, ratings_test):                           # Setting test set
        test_set = []
        for i in range(len(ratings_test)):                      # Selected ratings
            x = self.user_id_index[ratings_test.iloc[i,1]]      # Getting R indice for the given user_id and movie_id
            y = self.item_id_index[ratings_test.iloc[i,0]]
            z = ratings_test.iloc[i,2]
            test_set.append([x, y, z])
            self.R[x, y] = 0                    # Setting test set ratings to 0
        self.test_set = test_set
        return test_set                         # Return test set

    def test(self):                             # Training 하면서 test set의 정확도를 계산하는 메소드 
        # Initializing user-feature and movie-feature matrix
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        # List of training samples
        rows, columns = self.R.nonzero()
        self.samples = [(i,j, self.R[i,j]) for i, j in zip(rows, columns)]

        # Stochastic gradient descent for given number of iterations
        best_RMSE = 10000
        best_iteration = 0
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 1 == 0:
                    print("Iteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i+1, rmse1, rmse2))
            if best_RMSE > rmse2:                      # New best record
                best_RMSE = rmse2
                best_iteration = i
            elif (rmse2 - best_RMSE) > self.tolerance: # RMSE is increasing over tolerance
                break
        print(best_iteration, best_RMSE)
        return training_process

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])

    # Computing mean squared error
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    # Test RMSE 계산하는 method 
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)
        return np.sqrt(error/len(self.test_set))

    # Ratings for user i and moive j
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Ratings for user_id and moive_id
    def get_one_prediction(self, user_id, movie_id):
        return self.get_prediction(self.user_id_index[user_id], self.item_id_index[movie_id])

In [44]:
# Creating MF Object & train
ratings_temp = ratings.copy()
mf = NEW_MF(ratings_temp, K=200, alpha=0.0014, beta=0.075, iterations=350, tolerance=0.0000001, verbose=True) # tolerance 0.0000001로 줄임 1e -7
test_set = mf.set_test(ratings_test)
result = mf.test()

Iteration: 1 ; Train RMSE = 0.978824 ; Test RMSE = 1.006526
Iteration: 2 ; Train RMSE = 0.971435 ; Test RMSE = 1.001717
Iteration: 3 ; Train RMSE = 0.964778 ; Test RMSE = 0.997503
Iteration: 4 ; Train RMSE = 0.958665 ; Test RMSE = 0.993728
Iteration: 5 ; Train RMSE = 0.952981 ; Test RMSE = 0.990292
Iteration: 6 ; Train RMSE = 0.947653 ; Test RMSE = 0.987129
Iteration: 7 ; Train RMSE = 0.942632 ; Test RMSE = 0.984201
Iteration: 8 ; Train RMSE = 0.937880 ; Test RMSE = 0.981477
Iteration: 9 ; Train RMSE = 0.933372 ; Test RMSE = 0.978936
Iteration: 10 ; Train RMSE = 0.929083 ; Test RMSE = 0.976550
Iteration: 11 ; Train RMSE = 0.924993 ; Test RMSE = 0.974313
Iteration: 12 ; Train RMSE = 0.921092 ; Test RMSE = 0.972213
Iteration: 13 ; Train RMSE = 0.917359 ; Test RMSE = 0.970228
Iteration: 14 ; Train RMSE = 0.913786 ; Test RMSE = 0.968357
Iteration: 15 ; Train RMSE = 0.910363 ; Test RMSE = 0.966590
Iteration: 16 ; Train RMSE = 0.907079 ; Test RMSE = 0.964917
Iteration: 17 ; Train RMSE = 0.90

In [45]:
###################################### UBCF_BIAS_KNN ##################################################
# Predictions using CF ###########################################################################
from sklearn.metrics.pairwise import cosine_similarity
rating_matrix = ratings_train.pivot_table(values='rating', index='user_id', columns='item_id')

# 모든 가능한 사용자 pair의 Cosine similarities 계산
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
# 모든 user의 rating 평균 계산 
rating_mean = rating_matrix.mean(axis=1)

def ubcf_bias_knn(user_id, movie_id, neighbor_size=0):
    # 현 user의 평균 가져오기
    user_mean = rating_mean[user_id]
    if movie_id in rating_matrix:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id]
        # 현 movie의 rating 가져오기
        movie_ratings = rating_matrix[movie_id]
        # 모든 사용자의 rating 평균 가져오기
        others_mean = rating_mean
        # 현 movie에 대한 rating이 없는 user 삭제
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        others_mean = others_mean.drop(none_rating_idx)
        if neighbor_size == 0:               # Neighbor size가 지정되지 않은 경우
            # 편차로 예측치 계산
            movie_ratings = movie_ratings - others_mean
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 예측값에 현 사용자의 평균 더하기
            prediction = prediction + user_mean
        else:                                # Neighbor size가 지정된 경우
            # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
            neighbor_size = min(neighbor_size, len(sim_scores))
            # array로 바꾸기 (argsort를 사용하기 위함)
            sim_scores = np.array(sim_scores)
            movie_ratings = np.array(movie_ratings)
            others_mean = np.array(others_mean)
            # 유사도를 순서대로 정렬
            user_idx = np.argsort(sim_scores)
            # 유사도와 rating을 neighbor size만큼 받기
            sim_scores = sim_scores[user_idx][-neighbor_size:]
            movie_ratings = movie_ratings[user_idx][-neighbor_size:]
            # 사용자의 mean을 neighbor size만큼 받기
            others_mean = others_mean[user_idx][-neighbor_size:]
            # 편차로 예측치 계산
            movie_ratings = movie_ratings - others_mean
            if sim_scores.sum() >= 0.000001:
                # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + user_mean
            else:
                prediction = user_mean
    else:
        prediction = user_mean
    return prediction

In [46]:
###################################### DL ##################################################
# Predictions using DL ###########################################################################
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.layers import Dropout, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam, Adamax
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping  # 최고의 성능일 때 모델을 쓰기 위해

# train test 분리
from sklearn.utils import shuffle
#TRAIN_SIZE = 0.7
#df = shuffle(df)#, random_state=randNum)
#cutoff = int(TRAIN_SIZE * len(df))
df_train = ratings_train.copy() #df.iloc[:cutoff]
df_test = ratings_test.copy() #df.iloc[cutoff:]

df_train['user_id'] = df_train['user_id'].astype('category')
df_test['user_id'] = df_test['user_id'].astype('category')
df_train['item_id'] = df_train['item_id'].astype('category')
df_test['item_id'] = df_test['item_id'].astype('category')
df_train['user_id'] = df_train['user_id'].cat.codes       # 일련번호로
df_test['user_id'] = df_test['user_id'].cat.codes       # 일련번호로
df_train['item_id'] = df_train['item_id'].cat.codes
df_test['item_id'] = df_test['item_id'].cat.codes

# Variable 초기화 
K = 200                             # Latent factor 수 
reg = 0.0002                        # Regularization penalty
mu = df_train.rating.mean()    # 전체 평균 
N = len(set(df.user_id)) + 1   # Number of users
M = len(set(df.item_id)) + 1  # Number of items

# Defining RMSE measure
def RMSE(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

# Keras model
user = Input(shape=(1, ))                                               # User input
item = Input(shape=(1, ))                                               # Item input
P_embedding = Embedding(N, K, embeddings_regularizer=l2(reg))(user)     # (N, 1, K)
Q_embedding = Embedding(M, K, embeddings_regularizer=l2(reg))(item)     # (M, 1, K)
user_bias = Embedding(N, 1, embeddings_regularizer=l2(reg))(user)       # User bias term (N, 1, )
item_bias = Embedding(M, 1, embeddings_regularizer=l2(reg))(item)       # Item bias term (M, 1, )
P_embedding = Flatten()(P_embedding)                                    # (N, K)
Q_embedding = Flatten()(Q_embedding)                                    # (N, K)
user_bias = Flatten()(user_bias)                                        # (N, K)
item_bias = Flatten()(item_bias) 

R = Concatenate()([P_embedding, Q_embedding, user_bias, item_bias])     # (N, 2K + 2)

# Neural network
R = Dense(2048)(R)
R = Activation('linear')(R)

# Adding more layers
R = Dense(1024)(R)
R = Activation('LeakyReLU')(R)
R = Dense(512)(R)
R = Activation('linear')(R)

R = Dense(1)(R)

model = Model(inputs=[user, item], outputs=R)
model.compile(
  loss=RMSE,
  optimizer=SGD(learning_rate=0.004, momentum=0.9),
  #optimizer=Adamax(lr=0.0005),
  metrics=[RMSE],
)
# model.summary()

#early_stopping = EarlyStopping(mode= 'auto')

result = model.fit(
  x=[df_train.user_id.values, df_train.item_id.values],
  y=df_train.rating.values - mu,
  epochs=40,
  batch_size=128,
  #callbacks=[early_stopping],
  validation_data=(
    [df_test.user_id.values, df_test.item_id.values],
    df_test.rating.values - mu
  )
)

# 정확도(RMSE)를 계산하는 함수 
def RMSE2(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 정확도(RMSE)를 계산하는 함수 
def RMSE2(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

user_ids = df_test.user_id.values
item_ids = df_test.item_id.values
y_pred = model.predict([user_ids, item_ids]) + mu
y_pred = np.ravel(y_pred, order='C')
y_true = np.array(df_test.rating)

RMSE2(y_true, y_pred)

Epoch 1/40
546/546 [==============================] - 17s 30ms/step - loss: 1.3159 - RMSE: 0.9812 - val_loss: 1.3396 - val_RMSE: 1.0080
Epoch 2/40
546/546 [==============================] - 18s 32ms/step - loss: 1.3078 - RMSE: 0.9788 - val_loss: 1.3340 - val_RMSE: 1.0081
Epoch 3/40
546/546 [==============================] - 17s 31ms/step - loss: 1.2980 - RMSE: 0.9748 - val_loss: 1.3294 - val_RMSE: 1.0091
Epoch 4/40
546/546 [==============================] - 17s 31ms/step - loss: 1.2831 - RMSE: 0.9653 - val_loss: 1.3332 - val_RMSE: 1.0183
Epoch 5/40
546/546 [==============================] - 17s 32ms/step - loss: 1.2596 - RMSE: 0.9473 - val_loss: 1.3366 - val_RMSE: 1.0270
Epoch 6/40
546/546 [==============================] - 17s 31ms/step - loss: 1.2231 - RMSE: 0.9159 - val_loss: 1.3530 - val_RMSE: 1.0485
Epoch 7/40
546/546 [==============================] - 17s 32ms/step - loss: 1.1831 - RMSE: 0.8809 - val_loss: 1.3692 - val_RMSE: 1.0697
Epoch 8/40
546/546 [============================

1.2694704891836468

In [47]:
############################## HYBRID 끝판왕 (CF+MF+DL) ####################################
# Hybrid recommendation ###########################################################################

def recommender0(recomm_list, mf):
    id_pairs = zip(recomm_list[:, 0], recomm_list[:, 1])
    recommendations = np.array([mf.get_one_prediction(user, movie) for (user, movie) in id_pairs])    
    return recommendations

def recommender1(recomm_list, neighbor_size=0):
    id_pairs = zip(recomm_list[:, 0], recomm_list[:, 1])
    recommendations = np.array([ubcf_bias_knn(user, movie, neighbor_size) for (user, movie) in id_pairs])
    return recommendations

def recommender2(recomm_list, model):
    user_ids = recomm_list[:,0]
    item_ids = recomm_list[:,1]
    recommendations = model.predict([user_ids, item_ids]) + mu
    return recommendations

# RMSE 계산을 위한 함수
def RMSE2(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [48]:
# MF와 ubcf_bias_knn의 recomm_list
recomm_list = np.array(ratings_test.iloc[:, [1, 0]])
# MF 성능계산
result0 = recommender0(recomm_list, mf) #여기서 최적값을 안불러오는데 이유가 뭘까??
print('MF의 성능은')
print(RMSE2(ratings_test['rating'], result0))

MF의 성능은
0.9363949352303772


In [49]:
# ubcf_bias_knn 최적의 k 값 찾기 + 성능 계산
a = 1.5
for K in range(15, 45, 3):
    result1 = recommender1(recomm_list, K)
    best_score = RMSE2(ratings_test['rating'], result1)
    if a > best_score:
        a = best_score
        best_knn = K
    else:
        pass
# best knn 값으로 최종 적용
result1 = recommender1(recomm_list, best_knn) # best_knn은 위에서 최적의 k값 도출한 거.
print('ubcf_bias_knn의 성능은')
print(RMSE2(ratings_test['rating'], result1))

ubcf_bias_knn의 성능은
1.0689668946115976


In [50]:
# DL의 recomm_list
recomm_list = np.array(df_test.iloc[:, [1, 0]])
# DL 성능계산
print('DL의 성능은')
result2 = np.ravel(recommender2(recomm_list, model))
print(RMSE2(ratings_test['rating'], result2))

DL의 성능은
936/936 [==============================] - 4s 5ms/step
1.2694704891836468


In [53]:
## 3중 for문
box_w = [0.75, 0.76, 0.77, 0.78, 0.79, 0.80, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
box_x = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.11, 0.12, 0.13, 0.14, 0.15]
box_y = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.11, 0.12]
best_RMSE = 10 #그냥 큰 수치 임의로 지정
for x in box_w: # x는 MF
    for y in box_x: # y는 CF
        for z in box_y: # z는 DL
            if x+y+z == 1:
                try:
                    weight = [x,y,z]
                    predictions = []
                    for i, number in enumerate(result0):
                            predictions.append(result0[i] * weight[0] + result1[i] * weight[1]+ result2[i] * weight[2])
                    #print("Weights - %.2f : %.2f : %.2f ; RMSE = %.7f" % (weight[0], weight[1], weight[2], RMSE2(ratings_test['rating'], predictions)))
                    rmse2 = RMSE2(ratings_test['rating'], predictions)
                    if best_RMSE > rmse2:                      # New best record
                        best_RMSE = rmse2
                        best_ratio = " 최고의 황금비율 \n MF : '%.2f' \n ubcf_bias_knn : '%.2f' \n DL : '%.2f' \n RMSE = %.7f" % (weight[0], weight[1], weight[2], RMSE2(ratings_test['rating'], predictions))
                except:
                    pass
print(best_ratio)

 최고의 황금비율 
 MF : '0.89' 
 ubcf_bias_knn : '0.08' 
 DL : '0.03' 
 RMSE = 0.9352961
